In [1]:
import numpy as np
import pandas as pd
import os
import regex as reg
import geopandas as gpd
from sklearn.preprocessing import LabelEncoder

In [6]:
#installing drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extract GSN data and puts it in 2 arrays; 1 for wins and 1 for positions

In [8]:
winner_list_gsn = np.array([])
all_encoded_data_1 = np.array([])
all_encoded_data_2 = np.array([])
gsn_game_dataframe = pd.DataFrame()
all_encoded_data_gsn = np.array([])
both_player_encoded_data = np.array([])

#directory = os.listdir('/files/') to run locally

directory = os.listdir('/content/extracted_files/files/')
gsn_files = [file for file in directory if file.endswith('.gsn')]

for gsn_file in gsn_files:
    with open('/content/extracted_files/files/' + str(gsn_file), 'r') as file:
        gsn_data = file.read()

    player_1_side_str = gsn_data.split('\n')[2][0:40]
    ##split the starting positions per player
    player_2_side_str = gsn_data.split('\n')[2][60:100]

    player_1_side = np.array(list(player_1_side_str))
    player_2_side = np.array(list(player_2_side_str)) #create arrays for encoding the starting positions

    print(gsn_file)
    encoder = LabelEncoder()
    encoded_data_1 = encoder.fit_transform(list(player_1_side)) #create dummy variables out of letters
    encoded_data_2 = encoder.fit_transform(list(player_2_side))

    if len(all_encoded_data_1) == 0:
        all_encoded_data_1 = encoded_data_1
        all_encoded_data_2 = encoded_data_2
    else:
        all_encoded_data_1 = np.vstack((all_encoded_data_1, encoded_data_1)) #add encoding to long array of all encoding for all games
        all_encoded_data_2 = np.vstack((all_encoded_data_2, encoded_data_2))
    winner_list_gsn = np.append(winner_list_gsn, int(gsn_data.split('\n')[-2][-1])) #creates a list of who won which game 1 indicates player 2 and 0 is the red player(player 1)

inverted_winner_list = 1 - winner_list_gsn
all_encoded_data_gsn = np.vstack((all_encoded_data_1 , all_encoded_data_2)) #creates a total numpy array with the first players position being the first 9021 arrays
both_sided_winner_list_gsn = np.append(inverted_winner_list, winner_list_gsn) # creates double the size of the winner list # doubles the size of the winner list by inverting the list

FileNotFoundError: [Errno 2] No such file or directory: '/content/extracted_files/files/'

In [ ]:
len(both_sided_winner_list_gsn), len(all_encoded_data_gsn)

Extract GSN data and puts it in 2 arrays; 1 for wins and 1 for positions

In [ ]:
winner_list_xml = np.array([])
all_encoded_data_1 = np.array([])
all_encoded_data_2 = np.array([])
xml_game_dataframe = pd.DataFrame()
all_encoded_data_xml = np.array([])
both_player_encoded_data = np.array([])


directory = os.listdir('Files/')
xml_files = [file for file in directory if file.endswith('.xml')]

for xml_file in xml_files:
    with open('Files/' + str(xml_file), 'r') as file:
        xml_data = file.read()

    if int(xml_data.split()[-3].split('"')[1]) == 3: #check if it is not a tie
        #total matches including ties was 85292 without its 82216
        continue
    else:
        player_1_side_str = xml_data.split()[7].split('"')[1][0:40]
        ##split the starting positions per player
        player_2_side_str = xml_data.split()[7].split('"')[1][60:100]

        player_1_side = np.array(list(player_1_side_str))
        player_2_side = np.array(list(player_2_side_str)) #create arrays for encoding the starting positions


        encoder = LabelEncoder()
        encoded_data_1 = encoder.fit_transform(list(player_1_side)) #create dummy variables out of letters
        encoded_data_2 = encoder.fit_transform(list(player_2_side))

        if len(all_encoded_data_1) == 0:
            all_encoded_data_1 = encoded_data_1
            all_encoded_data_2 = encoded_data_2
        else:
            all_encoded_data_1 = np.vstack((all_encoded_data_1, encoded_data_1)) #add encoding to long array of all encoding for all games
            all_encoded_data_2 = np.vstack((all_encoded_data_2, encoded_data_2))
        winner_list_xml = np.append(winner_list_xml, int(xml_data.split()[-3].split('"')[1])) #creates a list of who won which game 1 indicates player 2 and 0 is the red player(player 1)


i=-1
for num in winner_list_xml:
    i+=1
    if num ==  2: ##means player 1 lost we change it to 0
        winner_list_xml[i] = 0

inverted_winner_list_xml = 1 - winner_list_xml
all_encoded_data_xml = np.vstack((all_encoded_data_1 , all_encoded_data_2)) #creates a total numpy array with the first players position being the first 9021 arrays
both_sided_winner_list_xml = np.append(winner_list_xml, inverted_winner_list_xml) # creates double the size of the winner list # doubles the size of the winner list by inverting the list

In [ ]:
len(both_sided_winner_list_xml)

In [ ]:
len(all_encoded_data_xml)

Writes the data to a csv file and combines the GSN and XML files

In [ ]:
all_encoded_data = np.vstack((all_encoded_data_gsn, all_encoded_data_xml))
all_winner_list = np.append(both_sided_winner_list_gsn, both_sided_winner_list_xml)

In [ ]:
len(all_encoded_data), len(all_winner_list)

In [ ]:
np.savetxt("X_encoded_positions.csv", all_encoded_data, delimiter=",")
np.savetxt("Y_winners_encoded.csv", all_encoded_data, delimiter=",")
